# Extraction of Time Series in Three Regions with High Density of Wind Farms

In [2]:
import numpy as np
import os

In [3]:
path_in = '../../data'
path_out = '../../outputs/timeseries'

season = 'djf'  # 'djf' or 'jja'
regions = ['r1', 'r2', 'r3']
datasets = ['merra2', 'cfsv2', 'seas5']

seas5_arr = np.load(os.path.join(path_in, f'seas5_{season}.npz'))['W10M']
cfsv2_arr = np.load(os.path.join(path_in, f'cfsv2_{season}.npz'))['W10M']
merra2_arr = np.load(os.path.join(path_in, f'merra2_{season}.npz'))['W10M']
lon_arr = np.load(os.path.join(path_in, f'merra2_{season}.npz'))['longitude']
lat_arr = np.load(os.path.join(path_in, f'merra2_{season}.npz'))['latitude']

lat_indices = {
    "r3": np.array([65, 65, 66]),
    "r2": np.array([62, 63]), 
    "r1": np.array([58, 58, 59]),
}

lon_indices = {
    "r3": np.array([31, 32, 31]),
    "r2": np.array([34, 34]),
    "r1": np.array([37, 38, 37])
}

def extract_region_data(seas5, cfsv2, merra2, lon, lat, lon_indices, lat_indices):
    """
    Extracts data from SEAS5, CFSV2, and MERRA2 for the specified regions.

    Parameters:
        seas5: numpy.ndarray of shape (year, ensemble_member, lat, lon)
        cfsv2: numpy.ndarray of shape (year, ensemble_member, lat, lon)
        merra2: numpy.ndarray of shape (year, lat, lon)
        region_indices: dict containing latitude and longitude indices for each region

    Returns:
        A dictionary with extracted data.
    """
    result = {}

    result['seas5'] = np.moveaxis(seas5[:, :, lat_indices, lon_indices], -1, 1)
    result['cfsv2'] = np.moveaxis(cfsv2[:, :, lat_indices, lon_indices], -1, 1)
    result['merra2'] = merra2[:, lat_indices, lon_indices]
    result['lon'] = lon[lon_indices]
    result['lat'] = lat[lat_indices]

    return result

for region in regions:
    data_region = extract_region_data(
        seas5_arr, cfsv2_arr, merra2_arr, 
        lon_arr, lat_arr, 
        lon_indices[region], lat_indices[region]
        )
    
    file_name_out = os.path.join(path_out, f'timeseries_{season}_{region}.npz')
    np.savez(file_name_out, **data_region)
    